<a href="https://colab.research.google.com/github/dimup/NER_BERTandKoBERT/blob/main/BERT_NER_KMOU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **구글 드라이브와 연동**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<br>

# **개체명 인식 with BERT & Hugging Face**



1. Hugging Face의 PyTorch BERT를 사용하였습니다.
2. 윤주성님의 Pytorch-BERT-CRF-NER을 참고했습니다. (https://github.com/eagle705/pytorch-bert-crf-ner)
3. Naver review 감성 분류 자료를 참고했습니다. (https://colab.research.google.com/drive/1tIf0Ugdqg4qT7gcxia3tL7und64Rv1dP#scrollTo=P58qy4--s5_x)
<br>




<br>
<br>

# **install & import**
tranformers 버전이 최근 갱신되며 기존 코드가 적용되지 않는 오류가 있습니다.
<br>
따라서 3.4.0 버전으로 install을 진행합니다.

In [ ]:
# Hugging Face의 transformers 모델을 설치
!pip install "transformers==3.4.0"

     |████████████████████████████████| 1.3MB 8.2MB/s 
     |████████████████████████████████| 1.2MB 15.2MB/s 
     |████████████████████████████████| 890kB 45.8MB/s 
     |████████████████████████████████| 2.9MB 51.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=1a3bde93edf1de7b9ce249db11863f46c51fdee002a26614b75042e1cb8757cf
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import tensorflow as tf
import torch
import re
import pprint
from transformers import BertTokenizer
from transformers import BertForTokenClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

<br>
<br>

# **데이터 전처리**

### **Input, Target 데이터 파싱 및 가공**
한국 해양대학교의 개체명 코퍼스를 파싱하는 Data_Parsing_KMOU.ipynp 파일에서 진행
<br>

데이터 파싱 후, input과 target 데이터에 특수기호 따옴표 ‘  “ 는 토크나이저가 인식하지 못하는 오류가 있어. 이를 키보드의 ' "으로 대체하는 코드를 작성했습니다

작업을 완료한 데이터를 final_inpt.txt 와 final_target.txt로 저장했습니다

### **Input, target 데이터 load**

In [ ]:
f = open("drive/My Drive/BERT_NER/final_input.txt", 'r')
input_list = f.readlines()
f.close()
print(input_list[0:10], "\n", len(input_list))

['여기서 우투수는 우타자를 상대로 0.279, 좌타자를 상대로 0.286의 피안타율을 기록했다.\n', "위쪽 절반은 빨갛게, 나머지 반은 하얗게 칠해진 단단한 공. 미국과 유럽, 일본 등 전 세계 수천만 명이 스마트폰 안에서 던지고 있는 '몬스터 볼'이다.\n", '안 전 대표는 친박(친박근혜)과 친문(친문재인)을 제외한 세력이 합치는 \'제3지대론\'에 대해 "총선 민심이 저희를 깨워 주셨는데 그걸 스스로 부정하는 건 민심에 반하는 것"이라고 일축했다.\n', '얼마 전 안산에 있는 지아네 집을 찾았다.\n', 'tvN 관계자는 1월31일 "술을 마시며 이야기하는 취중진담 콘셉트의 장점을 최대한 살리기 위해 이 같이 결정했다"며 "15세 이상 관람가 때보다 조금 더 편안하고 자유로운 분위기를 전달하기 위해 맞춤 편집한다"고 밝혔다.\n', '슈틸리케 감독은 이날 전주월드컵경기장에서 팬 사인회와 시축으로 팬들을 만나며 K리그 개막을 축하한다.\n', '2015 K리그 클래식은 차두리(서울), 김신욱(울산), 정대세(수원), 이동국 등 내로라하는 스타선수들이 개막 준비를 끝내고 축구팬들을 맞을 채비를 마쳤다.\n', '선불카드 하나로 4명이 무기명으로 그린피 면제 혜택을 받을 수도 있다.\n', "지난 5일 부산에서 열린 문 전 대표의 북콘서트에는 '미생'을 그린 만화가 윤태호, 가수 강산에ㆍ박기영, 방송인 김미화가 참석했다.\n", '반면 2경기 째 승리를 올리지 못한 선두 서울은 6승1무2패, 승점 19로 제자리걸음했다.\n'] 
 17694


In [ ]:
f = open("drive/My Drive/BERT_NER/final_target.txt", 'r')
target_list = f.readlines()
f.close()
print(target_list[0:10], "\n", len(target_list))

['여기서 우투수는 우타자를 상대로 <0.279:PNT>, 좌타자를 상대로 <0.286:PNT>의 피안타율을 기록했다.\n', "위쪽 절반은 빨갛게, 나머지 반은 하얗게 칠해진 단단한 공. <미국:LOC>과 <유럽:LOC>, <일본:LOC> 등 전 세계 <수천만 명:NOH>이 스마트폰 안에서 던지고 있는 '몬스터 볼'이다.\n", '<안:PER> 전 대표는 친박(친<박근혜:PER>)과 친문(친<문재인:PER>)을 제외한 세력이 합치는 \'<제3:NOH>지대론\'에 대해 "총선 민심이 저희를 깨워 주셨는데 그걸 스스로 부정하는 건 민심에 반하는 것"이라고 일축했다.\n', '얼마 전 <안산:LOC>에 있는 <지아:PER>네 집을 찾았다.\n', '<tvN:ORG> 관계자는 <1월31일:DAT> "술을 마시며 이야기하는 취중진담 콘셉트의 장점을 최대한 살리기 위해 이 같이 결정했다"며 "<15세:NOH> 이상 관람가 때보다 조금 더 편안하고 자유로운 분위기를 전달하기 위해 맞춤 편집한다"고 밝혔다.\n', '<슈틸리케:PER> 감독은 이날 <전주월드컵경기장:LOC>에서 팬 사인회와 시축으로 팬들을 만나며 <K리그:POH> 개막을 축하한다.\n', '<2015:DAT> <K리그 클래식:POH>은 <차두리:PER>(<서울:LOC>), <김신욱:PER>(<울산:LOC>), <정대세:PER>(<수원:LOC>), <이동국:PER> 등 내로라하는 스타선수들이 개막 준비를 끝내고 축구팬들을 맞을 채비를 마쳤다.\n', '선불카드 <하나:NOH>로 <4명:NOH>이 무기명으로 그린피 면제 혜택을 받을 수도 있다.\n', "지난 <5일:DAT> <부산:LOC>에서 열린 <문:PER> 전 대표의 북콘서트에는 '<미생:POH>'을 그린 만화가 <윤태호:PER>, 가수 <강산에:PER>ㆍ<박기영:PER>, 방송인 <김미화:PER>가 참석했다.\n", '반면 <2경기:NOH> 째 승리를 올리지 못한 선두 <서울:LOC>은 <6승:NOH><1무:NOH><2패:NOH>, 승점 <19:NOH

## **데이터 전처리 : Input용**


#### WordPiece tokenizing -> Embedding -> Padding
BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False) 코드를 통해 BERT의 WordPiece tokenizer를 사용했습니다.

In [ ]:
def preprocessing_input(input_list):
  # CLS와 SEP토큰 부착
  text_CLS = ["[CLS] " + str(txt) + " [SEP]" for txt in input_list]

  # 토크나이저 정의 및 토크나이징
  tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
  tokenized_texts = [tokenizer.tokenize(txt) for txt in text_CLS]
  # 입력 토큰의 최대 시퀀스 길이
  MAX_LEN = 128
 
  # 토큰 임베딩 수행
  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts] # token embedding. BERT의 token -> vector value로 바꾸는 vocab을 참조하는 것.
  num=1
  
  # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움 
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
 
  # 어텐션 마스크 초기화
  attention_masks = []

  # 어텐션 마스크, 값이 있다면 1, 패딩 부분이면 0으로 설정
  # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
  for seq in input_ids:
      seq_mask = [float(i>0) for i in seq]
      attention_masks.append(seq_mask)
 
  return input_ids, attention_masks, tokenized_texts

In [ ]:
input_ids, attention_masks, tokenized_texts = preprocessing_input(input_list)

## **데이터 전처리 : Target용**



#### WordPiece tokenizing

In [ ]:
def preprocessing_target(input_list, target_list):
  # 토크나이징
  tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
  tokenized_target_text = [tokenizer.tokenize(text_temp) for text_temp in target_list]
  # CLS, SEP가 부착되지 않은 input 데이터
  tokenized_texts_withoutCLS = [tokenizer.tokenize(text) for text in input_list]

  new_result_target_list = matching_input_target(tokenized_texts_withoutCLS, tokenized_target_text)
  prefix_sum_of_token_start_index_list, list_of_ner_tag_list, list_of_ner_text_list, list_of_tuple_ner_start_end_list = searching_boundary(tokenized_texts_withoutCLS, new_result_target_list)
  list_of_ner_label_list = tagging_label(tokenized_texts_withoutCLS, prefix_sum_of_token_start_index_list, list_of_ner_tag_list, list_of_ner_text_list, list_of_tuple_ner_start_end_list)
  ner_label_list_result = append_CLS_SEP(list_of_ner_label_list)
  return ner_label_list_result

대괄호 "[", "]" 등이 껴있는 target text는 WordPiece tokenizing 결과가 input text와 다르게 나오는 경우가 있어서, 이를 해결하기 위해 형태를 맞추는 작업을 진행합니다.





In [ ]:
def matching_input_target(tokenized_texts_withoutCLS, tokenized_target_text):
  # tokenizing된 input text들의 띄어쓰기를 제거하여 저장해놓음

  result_string_list_input = []
  for target_token in tokenized_texts_withoutCLS:
    result_string = ""
    for tkd in target_token:
      result_string += tkd
    result_string_list_input.append(result_string)
  result_string_list = []
    
  # list형식의 tokenized target token들을 하나의 문자열로 변환 
  for target_token in tokenized_target_text:
    result_string = ""
    for tkd in target_token:
      result_string += tkd
    result_string_list.append(result_string)

  regex_ner = re.compile('<(.+?):[A-Z#]{5}>')

  list_of_ner_all_list = []
  list_of_ner_text_list = []

  for result_string in result_string_list:
    filterd_by_regex = regex_ner.finditer(result_string)

    list_of_ner_all = []
    list_of_ner_text = []


    for match_item in filterd_by_regex:
        ner_all = match_item[0]  # <4일간:D##UR> -> DUR
        ner_text = match_item[1]  # <4일간:D##UR> -> 4일간

        list_of_ner_all.append(ner_all)
        list_of_ner_text.append(ner_text)

    list_of_ner_all_list.append(list_of_ner_all)
    list_of_ner_text_list.append(list_of_ner_text)

  result_target_list = result_string_list_input

  
  new_result_target_list = []
  for target, all, text in zip(result_target_list, list_of_ner_all_list, list_of_ner_text_list):
    is_same = ""
    for a, t in zip(all, text):
      if t == is_same:
        continue
      else:
        target = target.replace(t, a)
        is_same = t
    total_target=target.replace('##<', '<##')
    new_result_target_list.append(total_target)

  return new_result_target_list

### **Input에 대응되는 개체명 테그셋 만들기**

개체명이 나타나는 부분이 어디인지 알아내기 위한 사전준비로 다음을 수행합니다.


1.   하나의 input 문장을 이루는 token들이 각각 몇 번째에 출현하는지 찾아냄 
2.   이전에 문자열로 변환했던 target 문장에서 정규식을 통해 개체명, 개체명 태그, 개체명이 출현하는 범위를 찾아냄

In [ ]:
def searching_boundary(tokenized_texts_withoutCLS, new_result_target_list):
  prefix_sum_of_token_start_index_list = []
  for tokenized_text_w in tokenized_texts_withoutCLS:
    prefix_sum_of_token_start_index = []
    sum = 0
    for i, token in enumerate(tokenized_text_w):
        if i == 0:
            prefix_sum_of_token_start_index.append(0)
            sum += len(token)
        else:
            prefix_sum_of_token_start_index.append(sum)
            sum += len(token)

    prefix_sum_of_token_start_index_list.append(prefix_sum_of_token_start_index)

  regex_ner = re.compile('<(.+?):[A-Z#]{5}>')

  list_of_ner_tag_list = []
  list_of_ner_text_list = []
  list_of_tuple_ner_start_end_list = []

  for result_string in new_result_target_list:
    filterd_by_regex = regex_ner.finditer(result_string)

    list_of_ner_tag = []
    list_of_ner_text = []
    list_of_tuple_ner_start_end = []

    count_of_match = 0

    for match_item in filterd_by_regex:
        # 찾은 개체명 tag를 원본의 형태로 변환 ex) D##UR> -> DUR
        ner_tag_before = match_item[0][-6:-1]  
        ner_tag = ner_tag_before.replace("#","")
        ner_text = match_item[1]  # <4일간:D##UR> -> 4일간
        start_index = match_item.start() - 8 * count_of_match  # delete previous '<, :, 5 words tag name, >'
        end_index = match_item.end() - 8 - 8 * count_of_match

        list_of_ner_tag.append(ner_tag)
        list_of_ner_text.append(ner_text)
        list_of_tuple_ner_start_end.append((start_index, end_index))
        count_of_match += 1

    list_of_ner_tag_list.append(list_of_ner_tag)
    list_of_ner_text_list.append(list_of_ner_text)
    list_of_tuple_ner_start_end_list.append(list_of_tuple_ner_start_end)

  return prefix_sum_of_token_start_index_list, list_of_ner_tag_list, list_of_ner_text_list, list_of_tuple_ner_start_end_list

구해놓은 각 input토큰의 시작점과, 실제 개체명의 출현 범위를 비교하며 토큰에 대한 태그를 부착합니다.

In [ ]:
def tagging_label(tokenized_texts_withoutCLS, prefix_sum_of_token_start_index_list, list_of_ner_tag_list, list_of_ner_text_list, list_of_tuple_ner_start_end_list):
  list_of_ner_label_list = []
  for i in range(0, len(tokenized_texts_withoutCLS)):
    list_of_ner_label = []
    entity_index = 0
    is_entity_still_B = True
    for tup in zip(tokenized_texts_withoutCLS[i], prefix_sum_of_token_start_index_list[i]): # tokenize된 텍스트와, 각 토큰의 시작점이 tup에 저장됨
        token, index = tup
        if entity_index < len(list_of_tuple_ner_start_end_list[i]): # entity_index가 분석 개체명 개수보다 작을 경우
            start, end = list_of_tuple_ner_start_end_list[i][entity_index] # start, end 값은 entity_index 번째의 start, end 값
            if end <= index:  # 토큰을 하나하나 보다가, 인식된 개체명을 넘어가면       # end : 원본 데이터에서 인식된 개체명 엔티티가 끝나는 지점, index : 각 토큰들의 시작점
                is_entity_still_B = True # is_entity_still_B = true로 설정
                entity_index = entity_index + 1 if entity_index + 1 < len(list_of_tuple_ner_start_end_list[i]) else entity_index # 아래 과정을 더하여 다음 인식된 개체명으로 넘어감
                start, end = list_of_tuple_ner_start_end_list[i][entity_index]
            # 인식된 개체명 사이에 index가 도달했을 경우
            if start <= index and index < end: 
                entity_tag = list_of_ner_tag_list[i][entity_index] # 해당 개체명에 대한 태그를 entity_tag에 저장
                if is_entity_still_B is True: # 엔티티의 첫 시작점이라면
                    entity_tag = 'B-' + entity_tag # entity_tag 앞에 B-를 붙임
                    list_of_ner_label.append(entity_tag) # label set에 결과물을 넣음
                    is_entity_still_B = False # is_entity_still_B = False로 설정, 이제부터는 시작점이 아니므로
                else: # 시작점이 아닐 경우
                    entity_tag = 'I-' + entity_tag  # entity_tag 앞에 I-를 붙임
                    list_of_ner_label.append(entity_tag) # label set에 넣음
            else: # 인식된 개체명이 없을 경우
                is_entity_still_B = True # is_entity_still_B = True로 설정
                entity_tag = 'O' # entity_tag = O로 변경
                list_of_ner_label.append(entity_tag) # label set에 넣음

        else:
            entity_tag = 'O'
            list_of_ner_label.append(entity_tag)
    list_of_ner_label_list.append(list_of_ner_label)
  return list_of_ner_label_list

Tagset(label)에 [SEP]와 [CLS] 부착

In [ ]:
def append_CLS_SEP(list_of_ner_label_list):
  print(list_of_ner_label_list[0:3])
  for ner_label in list_of_ner_label_list:
    ner_label.append("[SEP]")
    ner_label.insert(0, "[CLS]")
  print(list_of_ner_label_list[0:3])
  return list_of_ner_label_list

In [ ]:
ner_label_list_result = preprocessing_target(input_list, target_list)

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PNT', 'I-PNT', 'I-PNT', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PNT', 'I-PNT', 'I-PNT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'B-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'B-NOH', 'I-NOH', 'I-NOH', 'I-NOH', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NOH', 'I-NOH', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]
[['[CLS]', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PNT', 'I-PNT', 'I-PNT', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PNT', 'I-PNT', 'I-P

### target labels embedding & padding작업

target label(tagset) 에 대한 임베딩, 패딩 작업입니다. 기존엔 임베딩이 필요없을 것이라 생각했지만, pytorch의 Tensor 자료구조는 문자열을 지원하지 않는다는 것을 확인했습니다. 그러므로 Hugging face의 embedding을 사용하려 했으나, 토큰에 대한 임베딩 값이 모두 동일하게 설정되는 문제가 발생했습니다.

따라서, tagset에 대한 {태그:숫자값} dictionary를 아래와 같이 임의로 작성했습니다.

In [ ]:
tag_dict = {'[CLS]': 0, '[SEP]': 1, 'O' : 2,
            'B-PER': 3, 'I-PER': 4, 
            'B-ORG': 5, 'I-ORG': 6,
            'B-LOC': 7, 'I-LOC': 8, 
            'B-POH': 9, 'I-POH': 10,
            'B-DAT': 11, 'I-DAT': 12,
            'B-TIM': 13, 'I-TIM': 14,
            'B-DUR': 15, 'I-DUR': 16,
            'B-MNY': 17, 'I-MNY': 18,
            'B-PNT': 19, 'I-PNT': 20, 
            'B-NOH': 21, 'I-NOH': 22}


{숫자값:태그}을 갖는 역순의 dict입니다. 파이썬 dictionary에서 value값으로 key값을 찾는 것은 반복문을 통해야하는 번거러움이 있기 때문에 새로운 dict를 생성하는 방법을 택했습니다.

In [ ]:
tag_dict_decode = inv_map = {v: k for k, v in tag_dict.items()}

label(tagset)에 대한 embedding을 생성한 dict를 통해 진행합니다.

In [ ]:
labels_ids = []

# tag_dict로 embedding 진행
for label_list in ner_label_list_result:
  labels_ids_embedded = [tag_dict.get(x) for x in label_list]  
  labels_ids.append(labels_ids_embedded)

padding을 진행합니다.

In [ ]:
MAX_LEN = 128

labels_ids = pad_sequences(labels_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# **Modeling**

#### 데이터 구조 변환 및 Training 준비

준비한 각 데이터를 Pytorch의 Tensor라는 자료구조 형식으로 변환합니다. Hugging face의 pytorch 기반 라이브러리를 사용하기 때문입니다.

In [ ]:
model_inputs = torch.tensor(input_ids)
model_labels = torch.tensor(labels_ids)
model_masks = torch.tensor(attention_masks)

In [ ]:
# 배치 사이즈 (학습 한 번에 들어갈 데이터의 사이즈)
batch_size = 32

# Pytorch의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터를 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
model_data = TensorDataset(model_inputs, model_masks, model_labels)
model_sampler = RandomSampler(model_data)
model_dataloader = DataLoader(model_data, sampler=model_sampler, batch_size=batch_size)

Model을 생성하기 전, GPU연산을 해야 하기 때문에 GPU에 대한 정보를 파악하고, GPU 디바이스 설정을 진행합니다.

In [ ]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')
    
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla T4


transformers 라이브러리를 활용해서 model을 생성합니다.

In [ ]:
# Token 분류를 위한 BERT 모델 생성
model = BertForTokenClassification.from_pretrained("bert-base-multilingual-cased", num_labels = 23, 
                                                   output_attentions = False, output_hidden_states = False)
model.cuda()

# **Training**

### Optimizer와 Hyper parameter 설정

In [ ]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 5

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(model_dataloader) * epochs

# 학습률을 조금씩 감소시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)



### Training 수행

In [ ]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(model_dataloader):

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        # 로스 구함
        loss = outputs[0]
        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()
        # 스케줄러로 학습률 감소
        scheduler.step()
        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(model_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 5 ========
Training...

  Average training loss: 0.39
  Training epcoh took: 0:06:31

======== Epoch 2 / 5 ========
Training...

  Average training loss: 0.20
  Training epcoh took: 0:06:32

======== Epoch 3 / 5 ========
Training...

  Average training loss: 0.16
  Training epcoh took: 0:06:32

======== Epoch 4 / 5 ========
Training...

  Average training loss: 0.15
  Training epcoh took: 0:06:32

======== Epoch 5 / 5 ========
Training...

  Average training loss: 0.13
  Training epcoh took: 0:06:32

Training complete!


training 완료한 모델을 저장합니다.

In [ ]:
torch.save(model, 'drive/My Drive/BERT_NER/BERTmodel_1,pt')

# **Validation**

### validation을 위한 데이터셋 불러오기 및 전처리

#### validation을 위한 Input, Target 데이터 load

In [ ]:
f = open("drive/My Drive/BERT_NER/valid_input_final.txt", 'r')
valid_input_list = f.readlines()
f.close()
print(valid_input_list[0:10], "\n", len(valid_input_list))

['현대건설은 예선A조 첫 경기서 GS칼텍스를 3-2(25-23 24-26 22-25 25-20 15-13)로 제압했다 ．\n', '대한항공 "한전 꿇어"\n', '한전은 경기가 시작되자마자 강력한 서브로 대한항공 수비진을 흔든 뒤 이병주, 최귀동의 공격을 앞세워 첫 세트를 따내며 불의의 일격을 가했다 .\n', '체육진흥투표권 수탁사업자인 스포츠토토㈜(www.sportstoto.co.kr)는 28일 개막한 2008 KOVO컵 IBK기업은행배 양산프로배구대회를 대상으로 새로운 방식으로 변경된 배구토토 매치게임을 본격 발매한다고 밝혔다 .\n', '●하이원컵 SBS 채리티여자오픈(강원도 정선 하이원골프장)\n', '1공태욱(김해건설공)\n', '니시코리(일본) 기권승 카라누시치(크로아티아), 루옌순(대만) 1(1-6 6-4 6-7 4-6)3 치폴라(이스테이트)\n', '공 감독은 "달리 어쩌겠나 .\n', '한전은 등록선수 9명으로 2008 양산프로배구대회(양산실내체육관)에 나왔다 .\n', '경기내용은 좋아지겠지만 문성민이 와도 금세 팀이 달라지는 것이 아니다 .\n'] 
 4700


In [ ]:
f = open("drive/My Drive/BERT_NER/valid_target_final.txt", 'r')
valid_target_list = f.readlines()
f.close()
print(valid_target_list[0:10], "\n", len(valid_target_list))

['<현대건설:ORG>은 예선<A조:ORG> 첫 경기서 <GS칼텍스:ORG>를 <3-2:NOH>(<25-23:NOH> <24-26:NOH> <22-25:NOH> <25-20:NOH> <15-13:NOH>)로 제압했다 ．\n', '<대한항공:ORG> "<한전:ORG> 꿇어"\n', '<한전:ORG>은 경기가 시작되자마자 강력한 서브로 <대한항공:ORG> 수비진을 흔든 뒤 <이병주:PER>, <최귀동:PER>의 공격을 앞세워 첫 세트를 따내며 불의의 일격을 가했다 .\n', '체육진흥투표권 수탁사업자인 <스포츠토토㈜:POH>(<www.sportstoto.co.kr:POH>)는 <28일:DAT> 개막한 <2008:DAT> <KOVO컵 IBK기업은행배 양산프로배구대회:POH>를 대상으로 새로운 방식으로 변경된 배구토토 매치게임을 본격 발매한다고 밝혔다 .\n', '●<하이원컵 SBS 채리티여자오픈:POH>(<강원도:LOC> <정선 하이원골프장:LOC>)\n', '1<공태욱:PER>(<김해건설공:ORG>)\n', '<니시코리:PER>(<일본:LOC>) 기권승 <카라누시치:PER>(<크로아티아:LOC>), <루옌순:PER>(<대만:LOC>) 1(<1-6:NOH> <6-4:NOH> <6-7:NOH> <4-6:NOH>)<3:NOH> <치폴라:PER>(<이스테이트:LOC>)\n', '<공:PER> 감독은 "달리 어쩌겠나 .\n', '<한전:ORG>은 등록선수 <9명:NOH>으로 <2008:DAT> <양산프로배구대회:POH>(<양산실내체육관:LOC>)에 나왔다 .\n', '경기내용은 좋아지겠지만 <문성민:PER>이 와도 금세 팀이 달라지는 것이 아니다 .\n'] 
 4700


#### validation을 위한 데이터 전처리 : Input용

In [ ]:
valid_input_ids, valid_masks_ids, tokenized_texts = preprocessing_input(valid_input_list)
print(valid_input_ids[0], len(valid_input_ids))
print(valid_masks_ids[0], len(valid_masks_ids))

[   101 104518  71439  31928  10892   9576  18471  10738  20626   9750
  86015  12424  70069 119287 119352  98500    124    118    123    113
  10258    118  10328  10233    118  10314  10306    118  10258  10258
    118  10197  10208    118  10249    114   9202   9672 119116  12490
  10066    102      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0] 4700
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

#### validation을 위한 데이터 전처리 : Target용

In [ ]:
valid_target = preprocessing_target(valid_input_list, valid_target_list)

[['B-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'B-NOH', 'I-NOH', 'I-NOH', 'O', 'B-NOH', 'I-NOH', 'I-NOH', 'B-NOH', 'I-NOH', 'I-NOH', 'B-NOH', 'I-NOH', 'I-NOH', 'B-NOH', 'I-NOH', 'I-NOH', 'B-NOH', 'I-NOH', 'I-NOH', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-ORG', 'I-ORG', 'I-ORG', 'O', 'B-ORG', 'I-ORG', 'O', 'O'], ['B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'I-PER', 'O', 'B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]
[['[CLS]', 'B-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'B-NOH', 'I-NOH', 'I-NOH', 'O', 'B-NOH', 'I-NOH', 'I-NOH', 'B-NOH', 'I-NOH', 'I-NOH', 'B-NOH', 'I-NOH', 'I-NOH', 'B-NOH', 'I-NOH', 'I-NOH', 'B-NOH', 'I-NOH', 'I-NOH', 'O', 'O', 'O', 'O', 'O

### Validation 수행

저장했던 Model을 불러옵니다.

In [ ]:
model = torch.load('drive/My Drive/BERT_NER/BERTmodel_1,pt')

GPU 디바이스를 설정합니다.

In [ ]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')
    
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla T4


준비된 vaildation input data에 대한 forward를 수행합니다.

In [ ]:
# 평가모드로 변경
model.eval()
# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
pred_labels = [] # 예측 태그셋을 담을 리스트
# 데이터로더에서 배치만큼 반복하여 가져옴
for i in range(0, len(valid_input_list)):
    valid_inputs = torch.tensor([valid_input_ids[i]])
    valid_masks = torch.tensor([valid_masks_ids[i]])

    # 배치를 GPU에 넣음
    b_input_ids = valid_inputs.to(device)
    b_input_mask = valid_masks.to(device)
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    logits = np.argmax(outputs[0].to('cpu').numpy(), axis=2)
    
    # join bpe split tokens
    tokens = tokenizer.convert_ids_to_tokens(b_input_ids.to('cpu').numpy()[0])
    new_labels = []
    for token, label_idx in zip(tokens, logits[0]):
        if token == '[PAD]': # 패딩 전까지의 출력만을 보기 위해
            break
        new_labels.append(tag_dict_decode[label_idx])

    pred_labels.append(new_labels)

토큰과 토큰 관계에서 Accuracy 측정 결과입니다.

In [ ]:
num, total = 0, 0
for i in range(0, len(pred_labels)):
  for pred, target in zip(pred_labels[i], valid_target[i]):
    # 고정으로 들어가 있는 [CLS], [SEP] 는 제외함
    if (pred == '[CLS]' and target == '[CLS]') or (pred == '[SEP]' and target == '[SEP]'):
      continue
    if pred == target:
      num +=1
  total += len(pred_labels[i])-2 # [CLS], [SEP] 태그 수 빼기

print(num/total*100,"%")

88.40340752954108 %


# **Testing**

#### 입력 문장에 대한 전처리를 수행

In [ ]:
# 입력 데이터 전처리
def convert_input_data(sentences):
    text_CLS = ["[CLS] " + str(txt) + " [SEP]" for txt in sentences]
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
    # 토크나이징
    tokenized_texts = [tokenizer.tokenize(sent) for sent in text_CLS]
    MAX_LEN = 128 # MAX_LEN 설정
    # 임베딩 및 패딩 진행
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
    # 어텐션 마스크 설정
    attention_masks = []
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)
    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

#### 테스트를 위해 forward()를 수행하고 결과를 반환

In [ ]:
# 문장 테스트
def test_sentences(sentences):
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
    # 평가모드로 변경
    model.eval()
    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)
    if torch.cuda.is_available():    
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask
                        )
    logits = np.argmax(outputs[0].to('cpu').numpy(), axis=2)
    
    # join bpe split tokens
    tokens = tokenizer.convert_ids_to_tokens(b_input_ids.to('cpu').numpy()[0])
    new_tokens, new_labels = [], []
    for token, label_idx in zip(tokens, logits[0]):
        if token == '[PAD]': # 패딩 전까지의 출력만을 보기 위해
            break
        new_labels.append(tag_dict_decode[label_idx])
        new_tokens.append(token)

    # return logits
    return new_labels, new_tokens

#### Test 결과 확인

In [ ]:
input_text = '상명대학교의 송재우는 두레미담에서 5000원 내고 점심을 먹었다.'
new_label, new_token = test_sentences([input_text])

for token, label in zip(new_token, new_label): 
    print("{}\t{}".format(label, token))

[CLS]	[CLS]
B-ORG	상
I-ORG	##명
I-ORG	##대학교
O	##의
B-PER	송
I-PER	##재
I-PER	##우
O	##는
B-POH	두
I-POH	##레
I-POH	##미
I-POH	##담
O	##에서
B-MNY	5000
I-MNY	##원
O	내
O	##고
O	점
O	##심을
O	먹
O	##었다
O	.
[SEP]	[SEP]


# **Inference**

문장을 입력할 경우 해당 문장에 포함된 개체명에 태그를 부착하여 출력하도록 가공했습니다.


#### 출력 결과 가공

In [ ]:
def inferencing(input_text, new_label, new_token):
  all_inf_list = [] # inf_list들로 이루어진 리스트
  inf_list = [] # [태그, 태그에 대응하는 단어]로 이루어진 리스트
  isB = False # B- 태그가 나왔을 경우 true로 설정
  tagword = "" # 토큰을 하나씩 연결하여 태그에 대응하는 하나의 단어로 만들기 위한 문자열

  for label, token in zip(new_label, new_token):
    # B- 태그가 나왔을 경우
    if "B-" in label:
      # 만일 B- 태그가 나온상태에서 또 B- 태그가 나왔을 경우 (개체명이 연속으로 등장했을 경우)
      if isB:
        inf_list.append(tagword) 
        all_inf_list.append(inf_list)
        inf_list=[]
        tagword = ""
      # 태그명을 inf_list에 추가 ex) ORG, PER
      inf_list.append(label[2:5])
      
      if "##" in token: # 만일 B- 태그에 대응하는 토큰이 ##으로 시작할 경우
        tagword += token.replace("##","") # '##' 지우고 tagword에 추가
      else: # 아닐 경우
        tagword += token # 그냥 tagword에 추가

      isB = True # B- 태그가 나왔으니 True로 설정
    
    # B- 태그가 나온 상태고, I- 태그가 나왔을 때 (B- 태그와 연결되지 않은 I- 태그는 비정상으로 판단)
    elif isB and "I-" in label:
      if "##" in token: # 만일 I- 태그에 대응하는 토큰이 ##으로 시작할 경우
        tagword += token.replace("##","") # '##' 지우고 tagword에 추가
      else: # 아닐 경우
        """
        앞에 띄어쓰기를 한 칸 추가하고 tagword에 추가
        <왜?>
        토크나이징 이후엔 띄어쓰기가 제거된 상태.
        Bert의 Wordpiece tokenize는 띄어쓰기 이후의 단어엔 ##이 붙지 않는 특성을 활용하여
        띄어쓰기가 포함된 개체명을 제대로 출력하기 위함 ex) 11시 30분, 2020년 8월 15일
        """
        tagword += token.replace(token, " "+token) 

    # B- 태그가 나온 상태고, O 태그가 나왔을 때 (개체명이 끝났을 때)
    elif isB and "O" in label:
      inf_list.append(tagword) # 지금까지의 tagword를 inf_list에 추가
      all_inf_list.append(inf_list) # inf_list를 all_inf_list에 추가
      inf_list=[] # 싹 다 초기화
      tagword = ""
      isB = False
  
  # 예쁘게 출력하기
  for text_tag in all_inf_list:
    if text_tag[1] in input_text:
      input_text = input_text.replace(text_tag[1], "<"+text_tag[1]+":"+text_tag[0]+">")

  print(input_text)

#### Inference 결과 확인

In [ ]:
input_text = ""
while True:
  input_text = input("문장을 입력하세요 >> ")
  if input_text == "exit()":
    print("종료합니다.")
    break
  print("")
  #print("")
  #print("<원본 문장>\n",  input_text)
  #print("")
  new_label, new_token = test_sentences([input_text])
  print("-변환 문장-")
  inferencing(input_text, new_label, new_token)
  print("\n====================================================================\n")

문장을 입력하세요 >> 상명대학교의 송재우는 두레미담에서 5000원 내고 점심을 먹었다.

-변환 문장-
<상명대학교:ORG>의 <송재우:PER>는 <두레미담:POH>에서 <5000원:MNY> 내고 점심을 먹었다.


문장을 입력하세요 >> exit()
종료합니다.
